# Pre-Processing Surface Velocity Measurements

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta

import dateutil.parser

In [ ]:
HEADER_DATE_FORMAT = "%d.%m.%Y %H:%M:%S" 
OUTPUT_DATE_FORMAT = "%Y_%m_%d_%H_%M_%S_%f"

Parse raw measurements file and convert to file with proper time stamps

In [ ]:
input_file_velocity_measurements = "DEFINE PATH TO MEASUREMENTS"
input_file_velocity_header = "DEFINE PATH TO HEADER"

output_file_velocity_measurements = "DEFINE OUTPUT PATH"
cut_off_date = dateutil.parser.isoparse("YYYY-MM-DDTHH:MM:SS") #TODO: define correctly

cut_off_date

Get meta information:

In [ ]:
def _get_date_from_header_line(line):
    
    date_extractor = r"\d\d\.\d\d\.\d\d\d\d \d\d:\d\d:\d\d"
    match = re.findall(date_extractor, line)
    date = datetime.strptime(match[0], HEADER_DATE_FORMAT)
    return date

In [ ]:
with open(input_file_velocity_header, "r") as sensor_information:
    
    lines = sensor_information.readlines()
    
    start = _get_date_from_header_line(lines[4])
    stop = _get_date_from_header_line(lines[5])
    duration = stop - start
    duration_in_s = duration.seconds
    
    print("Measurement start: ", start)
    print("Measurement stop: ", stop)
    print("Duration: ", duration, " (= ", duration_in_s, "s)", sep="")
    
    # get sampling rate
    sampling_rate_extractor = r"\d+ Hz"
    match = re.findall(sampling_rate_extractor, lines[9])
    sampling_rate_str = match[0]
    print("Sampling rate: ", sampling_rate_str)

    # get number of measurements
    number_extractor = r"\d+"
    match = re.findall(number_extractor, lines[2])
    number_measurements = int(match[0])
    print("Number of measurements: ", number_measurements)
        
    # small sanity check
    match = re.findall(number_extractor, sampling_rate_str)
    sampling_rate = int(match[0])
    
    samples_per_second = number_measurements / duration_in_s
    print("Sanity Check:")
    print("\tExpected number of samples: ", duration_in_s * sampling_rate)
    print("\tCalculated samples per second: ", samples_per_second)
    print("\tDivergence: ", (samples_per_second-sampling_rate) / sampling_rate, "%", sep="")

Read file and ignore unused columns:

In [ ]:
# get column headers
with open(input_file_velocity_header, "r") as sensor_information:
    
    lines = sensor_information.readlines()
    
    i = 0
    n = len(lines)
    
    for k in range(n):
        if "Data file format" in lines[k]:
            i = k
            break
    
    # ignore 2 successive lines
    i += 3

    column_names = list()

    for k in range(20):
        
        current_line = lines[i+k]
        line_split = current_line.replace("  ", "\t").split("\t")
        
        measurement_name = line_split[1].strip()
        unit = line_split[-1].strip()
        column_name = "%s [%s]" % (measurement_name, unit)
        
        column_names.append(column_name)
        

k = None        
velocity_data = pd.read_csv(input_file_velocity_measurements, names=column_names, delim_whitespace=True)

print("Column Names:")
for name in column_names:
    print("\t", name)

name = None 

In [ ]:
velocity_data

In [ ]:
pd.read_csv(input_file_velocity_measurements, names=column_names, delim_whitespace=True)

Convert timestamps:

In [ ]:
velocity_data

In [ ]:
(velocity_data[column_names[1]])

In [ ]:
first_timestamp = velocity_data[column_names[1]].iloc[0]

timestamp_converter = lambda timestamp: start + timedelta(seconds=(timestamp-first_timestamp))

velocity_data["Timestamp"] = velocity_data[column_names[1]].apply(timestamp_converter)
velocity_data.set_index("Timestamp", drop=True, inplace=True)
velocity_data

In [ ]:
x_axis_column_name = column_names[4]
y_axis_column_name = column_names[5]
z_axis_column_name = column_names[6]

## Outlier Removal

Augment data with differences:

In [ ]:
x_axis_difference = "x_diff"
y_axis_difference = "y_diff"
z_axis_difference = "z_diff"

velocity_data[x_axis_difference] = velocity_data[x_axis_column_name].diff()
velocity_data[y_axis_difference] = velocity_data[y_axis_column_name].diff()
velocity_data[z_axis_difference] = velocity_data[z_axis_column_name].diff()

velocity_data

In [ ]:
quantile_25 = velocity_data.quantile(.25)
quantile_75 = velocity_data.quantile(.75)

iqr = quantile_75 - quantile_25

lower_innter_fence = quantile_25 - 1.5 * iqr
lower_outer_fence = quantile_25 - 3 * iqr

upper_inner_fence = quantile_75 + 1.5 * iqr
upper_outer_fence = quantile_75 + 3 * iqr

In [ ]:
def _add_vertical_cut_off_date(axes):
    
    n = axes.shape[0]
    m = axes.shape[1]

    for i in range(n):
        for j in range(m):
            axes[i][j].axvline(cut_off_date, color="black", linestyle="--")


In [ ]:
figure, axes = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(15,15/1.62))
figure.tight_layout()

velocity_data[x_axis_column_name].plot(ax=axes[0,0], title="Velocity x")
velocity_data[y_axis_column_name].plot(ax=axes[1,0], title="Velocity y")
velocity_data[z_axis_column_name].plot(ax=axes[2,0], title="Velocity z")

velocity_data[x_axis_difference].plot(ax=axes[0,1], title="Velocity Difference x")
velocity_data[y_axis_difference].plot(ax=axes[1,1], title="Velocity Difference y")
velocity_data[z_axis_difference].plot(ax=axes[2,1], title="Velocity Difference z")

for row_number, column_name in zip(range(3), [x_axis_difference, y_axis_difference, z_axis_difference]):
    
    axes[row_number, 1].axhline(lower_innter_fence[column_name], color="green")
    axes[row_number, 1].axhline(lower_outer_fence[column_name], color="red")

    axes[row_number, 1].axhline(upper_inner_fence[column_name], color="green")
    axes[row_number, 1].axhline(upper_outer_fence[column_name], color="red")


_add_vertical_cut_off_date(axes)
plt.show()
plt.close()

row_number = None 
column_name = None 

Remove outliers:

In [ ]:
remove_mild_outliers = False
remove_extreme_outliers = True

for column, difference_colum in zip([x_axis_column_name, y_axis_column_name, z_axis_column_name], 
                                    [x_axis_difference, y_axis_difference, z_axis_difference]):

    if remove_extreme_outliers:
        lower_extreme_outliers = velocity_data[difference_colum] < lower_outer_fence[difference_colum]
        upper_extreme_outliers = velocity_data[difference_colum] > upper_outer_fence[difference_colum]

        velocity_data.loc[lower_extreme_outliers, column] = np.nan
        velocity_data.loc[upper_extreme_outliers, column] = np.nan
        
    if remove_mild_outliers:
        lower_mild_outliers = velocity_data[difference_colum] < lower_innter_fence[difference_colum]
        upper_mild_outliers = velocity_data[difference_colum] > upper_inner_fence[difference_colum]

        velocity_data.loc[lower_mild_outliers, column] = np.nan
        velocity_data.loc[upper_mild_outliers, column] = np.nan

        
column = None 
difference_colum = None 

In [ ]:
figure, axes = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(15,15/1.62))
figure.tight_layout()

velocity_data[x_axis_column_name].plot(ax=axes[0,0], title="Velocity x")
velocity_data[y_axis_column_name].plot(ax=axes[1,0], title="Velocity y")
velocity_data[z_axis_column_name].plot(ax=axes[2,0], title="Velocity z")

velocity_data[x_axis_column_name].diff().plot(ax=axes[0,1], title="Velocity Difference x")
velocity_data[y_axis_column_name].diff().plot(ax=axes[1,1], title="Velocity Difference y")
velocity_data[z_axis_column_name].diff().plot(ax=axes[2,1], title="Velocity Difference z")

for row_number, column_name in zip(range(3), [x_axis_difference, y_axis_difference, z_axis_difference]):
    
    axes[row_number, 1].axhline(lower_innter_fence[column_name], color="green")
    axes[row_number, 1].axhline(lower_outer_fence[column_name], color="red")

    axes[row_number, 1].axhline(upper_inner_fence[column_name], color="green")
    axes[row_number, 1].axhline(upper_outer_fence[column_name], color="red")

_add_vertical_cut_off_date(axes)
plt.show()
plt.close()

row_number = None 
column_name = None 

## Smoothed Data

Aggregate over one second

In [ ]:
velocity_data = velocity_data.resample("1s").mean()
velocity_data

In [ ]:
velocity_column = "velocity"

velocity_data[velocity_column] = (velocity_data[x_axis_column_name] ** 2 \
                                + velocity_data[y_axis_column_name] ** 2 \
                                + velocity_data[z_axis_column_name] ** 2).pow(.5)

In [ ]:
SECONDS_TO_AVERAGE = 30
SAMPLE_WINDOW_SIZE = SECONDS_TO_AVERAGE

smooth_x = velocity_data[x_axis_column_name].rolling(SAMPLE_WINDOW_SIZE).mean()
smooth_y = velocity_data[y_axis_column_name].rolling(SAMPLE_WINDOW_SIZE).mean()
smooth_z = velocity_data[z_axis_column_name].rolling(SAMPLE_WINDOW_SIZE).mean()
smooth_v = velocity_data[velocity_column].rolling(SAMPLE_WINDOW_SIZE).mean()

smooth_x = smooth_x.interpolate()
smooth_y = smooth_y.interpolate()
smooth_z = smooth_z.interpolate()
smooth_v = smooth_v.interpolate()

In [ ]:
lower_rolling_percentile = 5
upper_rolling_percentile = 95

quantiles = [lower_rolling_percentile, upper_rolling_percentile] 
quantiles_as_floats = np.array(quantiles) / 100

def _get_column_name(percentile, axis):
    return "rolling%d%s" % (percentile, axis)


rolling_quantiles = pd.DataFrame(data ={
    "Timestamp" : velocity_data.index, # deliberately NOT setting as index yet. 
    
    _get_column_name(lower_rolling_percentile, "x") : np.zeros(len(smooth_x)),
    _get_column_name(upper_rolling_percentile, "x") : np.zeros(len(smooth_x)),
    
    _get_column_name(lower_rolling_percentile, "y") : np.zeros(len(smooth_y)),
    _get_column_name(upper_rolling_percentile, "y") : np.zeros(len(smooth_y)),
    
    _get_column_name(lower_rolling_percentile, "z") : np.zeros(len(smooth_z)),
    _get_column_name(upper_rolling_percentile, "z") : np.zeros(len(smooth_z)),
    
    _get_column_name(lower_rolling_percentile, "v") : np.zeros(len(smooth_v)),
    _get_column_name(upper_rolling_percentile, "v") : np.zeros(len(smooth_v))
})


def _rolling_quantile_calculator(row, original_dataframe):

    current_row_index = int(row.name)
    
    current_sub_dataframe = original_dataframe.head(current_row_index)
    
    for axis, original_column_name in zip(["x", "y", "z", "v"], [x_axis_column_name, y_axis_column_name, z_axis_column_name, velocity_column]):

        calculated_quantiles = current_sub_dataframe[original_column_name].quantile(quantiles_as_floats)
                
        row[_get_column_name(quantiles[0], axis)] = calculated_quantiles.iloc[0]
        row[_get_column_name(quantiles[1], axis)] = calculated_quantiles.iloc[1]
    
    return row
        
rolling_quantiles = rolling_quantiles.apply(_rolling_quantile_calculator, axis=1, original_dataframe=velocity_data)
rolling_quantiles.set_index("Timestamp", inplace=True, drop=True)

rolling_quantiles

In [ ]:
figure, axes = plt.subplots(nrows=4, ncols=2, sharex=True, figsize=(15,15/1.62))
figure.tight_layout()

velocity_data[x_axis_column_name].plot(ax=axes[0,0], title="Velocity x")
velocity_data[y_axis_column_name].plot(ax=axes[1,0], title="Velocity y")
velocity_data[z_axis_column_name].plot(ax=axes[2,0], title="Velocity z")
velocity_data[velocity_column].plot(ax=axes[3,0], title="Velocity")

smooth_x.plot(ax=axes[0,0])
smooth_y.plot(ax=axes[1,0])
smooth_z.plot(ax=axes[2,0])
smooth_v.plot(ax=axes[3,0])

smooth_x.plot(ax=axes[0,1], title="Rolling Average Velocity x")
smooth_y.plot(ax=axes[1,1], title="Rolling Average Velocity y")
smooth_z.plot(ax=axes[2,1], title="Rolling Average Velocity z")
smooth_v.plot(ax=axes[3,1], title="Rolling Average Velocity")

rolling_quantiles[_get_column_name(lower_rolling_percentile, "x")].plot(ax=axes[0,0])
rolling_quantiles[_get_column_name(upper_rolling_percentile, "x")].plot(ax=axes[0,0])

rolling_quantiles[_get_column_name(lower_rolling_percentile, "y")].plot(ax=axes[1,0])
rolling_quantiles[_get_column_name(upper_rolling_percentile, "y")].plot(ax=axes[1,0])

rolling_quantiles[_get_column_name(lower_rolling_percentile, "z")].plot(ax=axes[2,0])
rolling_quantiles[_get_column_name(upper_rolling_percentile, "z")].plot(ax=axes[2,0])

rolling_quantiles[_get_column_name(lower_rolling_percentile, "v")].plot(ax=axes[3,0])
rolling_quantiles[_get_column_name(upper_rolling_percentile, "v")].plot(ax=axes[3,0])

_add_vertical_cut_off_date(axes)
plt.show()
plt.close()

Take smoothed data as measurements. Write to file:

In [ ]:
alskdjalskjdaslkdj

In [ ]:
final_data = velocity_data[:cut_off_date][velocity_column]
final_data.to_csv(output_file_velocity_measurements, date_format=OUTPUT_DATE_FORMAT)
final_data

In [ ]:
final_data = velocity_data[:cut_off_date][velocity_column]
final_data

In [ ]:
sorted_final = final_data.to_frame()
sorted_final = sorted_final.sort_values(by=velocity_column, ignore_index=True)
sorted_final.plot()

In [ ]:
figure, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

sorted_final.plot(ax=axes)

axes.axhline(0.73, color="red")
axes.axhline(0.62, color="red")




In [ ]:
final_data

In [ ]:
training_set_information = pd.read_csv("")
